<a href="https://colab.research.google.com/github/jee365/ESAA/blob/main/Mini_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (6,860 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and d

In [3]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
train=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/movies_train.csv')
test=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/movies_test.csv')
submission=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/submission.csv')

In [8]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           243 non-null    object 
 1   distributor     243 non-null    object 
 2   genre           243 non-null    object 
 3   release_time    243 non-null    object 
 4   time            243 non-null    int64  
 5   screening_rat   243 non-null    object 
 6   director        243 non-null    object 
 7   dir_prev_bfnum  107 non-null    float64
 8   dir_prev_num    243 non-null    int64  
 9   num_staff       243 non-null    int64  
 10  num_actor       243 non-null    int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 21.0+ KB


데이터 전처리

In [9]:
test.isna().sum() / 600

title             0.000000
distributor       0.000000
genre             0.000000
release_time      0.000000
time              0.000000
screening_rat     0.000000
director          0.000000
dir_prev_bfnum    0.226667
dir_prev_num      0.000000
num_staff         0.000000
num_actor         0.000000
dtype: float64

In [10]:
train[train['dir_prev_bfnum'].isna()]['dir_prev_num'].sum()

0

In [11]:
train['dir_prev_bfnum'].fillna(0, inplace = True)

In [12]:
test[test['dir_prev_bfnum'].isna()]['dir_prev_num'].sum()

0

In [13]:
test['dir_prev_bfnum'].fillna(0, inplace = True)

In [16]:
features = ['time', 'dir_prev_num', 'num_staff', 'num_actor']
target = ['box_off_num']

In [17]:
X_train, X_test, y_train = train[features], test[features], train[target]

k-fold 교차검증

In [19]:
from sklearn.model_selection import KFold

k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

In [20]:
import lightgbm as lgb

model = lgb.LGBMRegressor(random_state=0, n_estimators=1000)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.99956e+12
Early stopping, best iteration is:
[14]	valid_0's l2: 1.82271e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.36371e+12
Early stopping, best iteration is:
[44]	valid_0's l2: 3.19684e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.60852e+12
Early stopping, best iteration is:
[13]	valid_0's l2: 1.27294e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.00841e+12
Early stopping, best iteration is:
[19]	valid_0's l2: 2.8533e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.55452e+12
Early stopping, best iteration is:
[6]	valid_0's l2: 1.95655e+12


In [22]:
preds = []
for model in models:
    preds.append(model.predict(X_test))
len(preds)

5

In [23]:
kfoldLightGBM = submission.copy()

In [24]:
kfoldLightGBM['box_off_num'] = np.mean(preds, axis = 0)

In [50]:
kfoldLightGBM.to_csv('kfoldLightGBM.csv', index = False)

feature engineering

In [26]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['genre'] = le.fit_transform(train['genre'])

In [32]:
test['genre'] = le.transform(test['genre'])
features = ['time', 'dir_prev_num', 'num_staff', 'num_actor', 'dir_prev_bfnum', 'genre']

In [33]:
X_train, X_test, y_train = train[features], test[features], train[target]

In [34]:
model = lgb.LGBMRegressor(random_state=777, n_estimators=1000)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.83412e+12
Early stopping, best iteration is:
[15]	valid_0's l2: 1.71878e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 3.45188e+12
Early stopping, best iteration is:
[34]	valid_0's l2: 3.30873e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.3865e+12
Early stopping, best iteration is:
[28]	valid_0's l2: 1.18138e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.86779e+12
Early stopping, best iteration is:
[28]	valid_0's l2: 2.81054e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.88036e+12
Early stopping, best iteration is:
[5]	valid_0's l2: 2.02763e+12


In [35]:
X_test.head()

,time,dir_prev_num,num_staff,num_actor,dir_prev_bfnum,genre
0,125,2,304,3,3.005290e+05,2
1,113,4,275,3,3.427002e+05,5
2,115,3,419,7,4.206611e+06,4
3,116,2,408,2,6.913420e+05,10
4,110,1,380,1,3.173800e+04,1


In [36]:
preds = []
for model in models:
    preds.append(model.predict(X_test))
len(preds)

5

In [38]:
feLightGBM = submission.copy()

In [39]:
feLightGBM['box_off_num'] = np.mean(preds, axis = 0)

In [49]:
feLightGBM.to_csv('feLightGBM.csv', index = False)

Grid Search

In [40]:
from sklearn.model_selection import GridSearchCV

In [41]:
model = lgb.LGBMRegressor(random_state=0, n_estimators=1000)


params = {
    'learning_rate': [0.1, 0.01, 0.003],
    'min_child_samples': [20, 30]}

gs = GridSearchCV(estimator=model,
            param_grid=params,
            scoring='neg_mean_squared_error',
            cv = k_fold)

In [42]:
gs.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=LGBMRegressor(n_estimators=1000, random_state=0),
             param_grid={'learning_rate': [0.1, 0.01, 0.003],
                         'min_child_samples': [20, 30]},
             scoring='neg_mean_squared_error')

In [43]:
model = lgb.LGBMRegressor(random_state=0, n_estimators=1000, learning_rate= 0.003, min_child_samples=30)

models = []

for train_idx, val_idx in k_fold.split(X_train):
    x_t = X_train.iloc[train_idx]
    y_t = y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set=(x_val, y_val), early_stopping_rounds=100, verbose = 100))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 2.36115e+12
[200]	valid_0's l2: 2.03294e+12
[300]	valid_0's l2: 1.85734e+12
[400]	valid_0's l2: 1.78235e+12
[500]	valid_0's l2: 1.73582e+12
[600]	valid_0's l2: 1.69985e+12
[700]	valid_0's l2: 1.69131e+12
[800]	valid_0's l2: 1.69266e+12
Early stopping, best iteration is:
[706]	valid_0's l2: 1.6911e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 4.05348e+12
[200]	valid_0's l2: 3.72778e+12
[300]	valid_0's l2: 3.56828e+12
[400]	valid_0's l2: 3.47228e+12
[500]	valid_0's l2: 3.41566e+12
[600]	valid_0's l2: 3.37408e+12
[700]	valid_0's l2: 3.35227e+12
Early stopping, best iteration is:
[674]	valid_0's l2: 3.35095e+12
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 1.77995e+12
[200]	valid_0's l2: 1.46938e+12
[300]	valid_0's l2: 1.31453e+12
[400]	valid_0's l2: 1.24365e+12
[500]	valid_0's l2: 1.2228e+12
[600]	valid_0's l2: 1.22176e+12
Early s

In [44]:
preds = []
for model in models:
    preds.append(model.predict(X_test))

In [45]:
gs.best_score_

-2265702055875.8877

In [46]:
gslgbm = submission.copy()

In [47]:
gslgbm['box_off_num'] =  np.mean(preds, axis = 0)

In [48]:
gslgbm.to_csv('gslgbm.csv', index = False)